In [75]:
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint
from pathlib import Path

from keys import Trip_advisor_key
from keys import geoapify_key

In [76]:

# Defining the District dataframe

Districts_df = pd.DataFrame({
    'Location': ['Toronto Distillery District', 'Toronto GTA Area', 'Toronto South West', 'Toronto North West', 'Toronto Yorkville', 'Ottawa City Centre' , 'Niagara Falls'],
    'Lat': [43.6503, 43.6426, 43.642567, 43.6552, 43.6707 , 45.4236, 43.087586], 
    'Lng': [-79.3596, 79.3871, -79.387054, -79.4139,  -79.3930 , -75.7009, -79.081206]  
})


# Defining the Restaurant DataFrame

Restaurants_df = pd.DataFrame(columns=['Location', 'Restaurant Name', 'Postal Code', 'Restaurant Address', 'Lat', 'Lng'])

# Loop for districts
radius = 1000

params = {
    'categories': 'catering.restaurant',
    'apiKey': geoapify_key,
}

base_url = 'https://api.geoapify.com/v2/places'

for index, row in Districts_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']

    params['filter'] = f'circle:{longitude},{latitude},{radius}'
    params['bias'] = f'proximity:{longitude},{latitude}'

    response = requests.get(base_url, params=params).json()
    for restaurant in response.get('features', []):
        try:
            
            new_row = {
                            'Location': row['Location'],
                            'Restaurant Name': restaurant["properties"]["address_line1"],
                            'Postal Code' : restaurant["properties"]["postcode"],
                            'Restaurant Address': restaurant["properties"]["address_line2"],
                            'Lat': restaurant["geometry"]["coordinates"][1],
                            'Lng': restaurant["geometry"]["coordinates"][0]
                        }
            
            Restaurants_df.loc[len(Restaurants_df)] = new_row
        except(KeyError, IndexError):
            print('Error.')


In [79]:
Restaurants_df.head()

,Location,Restaurant Name,Postal Code,Restaurant Address,Lat,Lng
0,Toronto Distillery District,Archeo Trattoria,M5A 3C4,"31 Trinity Street, Old Toronto, ON M5A 3C4, Ca...",43.650682,-79.359630
1,Toronto Distillery District,El Catrin,M5A 3C4,"Mill Street, Old Toronto, ON M5A 3C4, Canada",43.650693,-79.359245
2,Toronto Distillery District,Oyster House & grill,M5A 3C4,"Tank House Lane, Old Toronto, ON M5A 3C4, Canada",43.650350,-79.358811
3,Toronto Distillery District,Pure Spirits,M5A 3C4,"17 Tank House Lane, Old Toronto, ON M5A 3C4, C...",43.650395,-79.358754
4,Toronto Distillery District,The Fermenting Cellar,M5A 3C4,"28 Distillery Lane, Old Toronto, ON M5A 3C4, C...",43.649806,-79.360538


In [4]:
# Export the Restaurant_df into a csv
Restaurants_df.to_csv("output_data/restaurants.csv", index_label="Restaurant ID")

In [ ]:
#Import restaurants.csv
Restaurants_df = pd.read_csv('output_data/restaurants.csv')

In [80]:
#Drill for location_id

Restaurants_df['Location ID'] = ''

err_count = 0


for index, restaurant in Restaurants_df.iterrows():
    
    url = f"https://api.content.tripadvisor.com/api/v1/location/search?key={Trip_advisor_key}&searchQuery={restaurant['Restaurant Name']}&language=en"

    headers = {"accept": "application/json"}

    response = requests.get(url, headers=headers).json()
    
    try:
        
        for res in response['data']:
    
            if res['address_obj']['postalcode'] == restaurant['Postal Code']:
            
                Restaurants_df.at[index, 'Location ID'] = res['location_id']
                
                break
    
    except Exception as e:
        
        err_count = err_count +1
        
        
#print(response.text)
print(err_count)
Restaurants_df.head()

49


,Location,Restaurant Name,Postal Code,Restaurant Address,Lat,Lng,Location ID
0,Toronto Distillery District,Archeo Trattoria,M5A 3C4,"31 Trinity Street, Old Toronto, ON M5A 3C4, Ca...",43.650682,-79.359630,5078107
1,Toronto Distillery District,El Catrin,M5A 3C4,"Mill Street, Old Toronto, ON M5A 3C4, Canada",43.650693,-79.359245,4556927
2,Toronto Distillery District,Oyster House & grill,M5A 3C4,"Tank House Lane, Old Toronto, ON M5A 3C4, Canada",43.650350,-79.358811,
3,Toronto Distillery District,Pure Spirits,M5A 3C4,"17 Tank House Lane, Old Toronto, ON M5A 3C4, C...",43.650395,-79.358754,708144
4,Toronto Distillery District,The Fermenting Cellar,M5A 3C4,"28 Distillery Lane, Old Toronto, ON M5A 3C4, C...",43.649806,-79.360538,


In [85]:
#Drop row with empty records
Restaurants_df = Restaurants_df[Restaurants_df['Location ID'] != '']
Restaurants_df

,Restaurant ID,Location,Restaurant Name,Postal Code,Restaurant Address,Lat,Lng,Location ID
0,0,Toronto Distillery District,Archeo Trattoria,M5A 3C4,"31 Trinity Street, Old Toronto, ON M5A 3C4, Ca...",43.650682,-79.359630,5078107
1,1,Toronto Distillery District,El Catrin,M5A 3C4,"Mill Street, Old Toronto, ON M5A 3C4, Canada",43.650693,-79.359245,4556927
2,3,Toronto Distillery District,Pure Spirits,M5A 3C4,"17 Tank House Lane, Old Toronto, ON M5A 3C4, C...",43.650395,-79.358754,708144
3,5,Toronto Distillery District,Cluny,M5A 3C4,"Tank House Lane, Old Toronto, ON M5A 3C4, Canada",43.650604,-79.357865,6762792
4,25,Toronto South West,Azure Restaurant & Bar,M5V 2X3,"225 Front Street West, Old Toronto, ON M5V 2X3...",43.644434,-79.385390,682125
5,26,Toronto South West,SOCO Kitchen + Bar,M5J 3A6,"75 Lower Simcoe Street, Old Toronto, ON M5J 3A...",43.642679,-79.383845,7243578
6,33,Toronto South West,Buster's Sea Cove,M5J 0A8,"PATH, Old Toronto, ON M5J 0A8, Canada",43.643164,-79.382862,19815927
7,43,Toronto North West,Café Diplomatico,M6G 1B3,"594 College Street, Old Toronto, ON M6G 1B3, C...",43.655369,-79.413731,683135
8,47,Toronto North West,Hapa Izakaya,M6G 1B4,"602 College Street, Old Toronto, ON M6G 1B4, C...",43.655255,-79.414264,3506716
9,57,Toronto North West,Duff's Famous Wings,M6G 1B1,"558 College Street, Old Toronto, ON M6G 1B1, C...",43.655711,-79.412207,1510697


In [86]:
# Export the Restaurant_df with restaurants Location ID into a csv 
Restaurants_df.to_csv("output_data/restaurants.csv", index = False)

In [83]:
#Drill for reviews 
Restaurants_df = pd.read_csv('output_data/restaurants.csv')

#New df Reviews
Reviews_df = pd.DataFrame(columns=['Location','Restaurant Name','Lat','Lng','Location ID','Review'])

#Looooppping!!! Again. Yes.
for index, restaurant in Restaurants_df.iterrows():
    
    url = f"https://api.content.tripadvisor.com/api/v1/location/{restaurant['Location ID']}/reviews?key={Trip_advisor_key}&language=en"
    
    headers = {"accept": "application/json"}
    
    response = requests.get(url, headers=headers).json()
    
    #Let's try this!!!
    try:
        
        for res in response['data']: 
            
            new_row = {'Location' : restaurant['Location'],
                       'Restaurant Name' : restaurant['Restaurant Name'],
                       'Lat': restaurant['Lat'],
                       'Lng': restaurant['Lng'],
                       'Location ID' : restaurant['Restaurant Name'],
                       'Review' : res['rating']}
            Reviews_df.loc[len(Reviews_df)] = new_row
    
    except(KeyError, IndexError):
            print('Error.')
            

In [84]:
Reviews_df.head(10)

,Location,Restaurant Name,Lat,Lng,Location ID,Review
0,Toronto Distillery District,Archeo Trattoria,43.650682,-79.359630,Archeo Trattoria,2
1,Toronto Distillery District,Archeo Trattoria,43.650682,-79.359630,Archeo Trattoria,3
2,Toronto Distillery District,Archeo Trattoria,43.650682,-79.359630,Archeo Trattoria,1
3,Toronto Distillery District,Archeo Trattoria,43.650682,-79.359630,Archeo Trattoria,5
4,Toronto Distillery District,Archeo Trattoria,43.650682,-79.359630,Archeo Trattoria,5
5,Toronto Distillery District,El Catrin,43.650693,-79.359245,El Catrin,5
6,Toronto Distillery District,El Catrin,43.650693,-79.359245,El Catrin,2
7,Toronto Distillery District,El Catrin,43.650693,-79.359245,El Catrin,3
8,Toronto Distillery District,El Catrin,43.650693,-79.359245,El Catrin,5
9,Toronto Distillery District,El Catrin,43.650693,-79.359245,El Catrin,4


In [22]:
Restaurants_df['Location ID'] = ''

restaurant_id = []

City_url = "https://tripadvisor16.p.rapidapi.com/api/v1/restaurant/searchLocation"

querystring = {"query":"toronto"}

headers = {
    "X-RapidAPI-Key": '02b64322b1msh1c9677a5e07e412p102c76jsn34cac83b4fd0',
    "X-RapidAPI-Host": "tripadvisor16.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

City_id = (response.json()["data"][0]["locationId"])

res_url = "https://tripadvisor16.p.rapidapi.com/api/v1/restaurant/searchRestaurants"

res_qr = {"locationId":City_id}

restaurants = requests.get(res_url, headers=headers, params=res_qr)

err_count = 0

for index, restaurant in Restaurants_df.iterrows():
    
    try:
        
        for i in range(len(resturants.json()["data"]["data"])):
            
            if restaurant['Restaurant Name'] == restaurants["data"]["data"][i]['name']:
                
                restaurant_id.append(resturants.json()["data"]["data"][i]["locationId"])
                
                break
                
                
    except Exception as e:
        
        err_count = err_count +1
        
    



#print(response.json())

In [74]:
Reviews_df.count()

Location           110
Restaurant Name    110
Location ID        110
Review             110
dtype: int64

In [ ]:
Locations = ['toronto', 'ottawa', 'niagara']

for loc in Locations:
    
    
    city = loc
    city_url = "https://tripadvisor16.p.rapidapi.com/api/v1/restaurant/searchLocation"

    city_querystring = {"query": city}

    headers = {
        "X-RapidAPI-Key": '02b64322b1msh1c9677a5e07e412p102c76jsn34cac83b4fd0',
        "X-RapidAPI-Host": "tripadvisor16.p.rapidapi.com"
    }

    response = requests.get(city_url, headers=headers, params= city_querystring)


    hey = []
    res_name = []
    res_add = []
    review_dates = []
    count = 0


    location_id = (response.json()["data"][0]["locationId"])

    loc_url = "https://tripadvisor16.p.rapidapi.com/api/v1/restaurant/searchRestaurants"


    loc_querystring = {"locationId":location_id}

    resturants = requests.get(loc_url, headers=headers, params=loc_querystring)


    for i in range(len(resturants.json()["data"]["data"])):
        hey.append(resturants.json()["data"]["data"][i]["restaurantsId"])


    det_url = "https://tripadvisor16.p.rapidapi.com/api/v1/restaurant/getRestaurantDetails"


   
    for res_id in hey:

        dets_querystring = {"restaurantsId":res_id,"currencyCode":"USD"}    
        details = requests.get(det_url, headers=headers, params=dets_querystring)  

        if 'data' in details.json():
        
            res_add.append(details.json()["data"]['location']["address"])
            res_name.append(details.json()["data"]['location']['name'])

    
            if int(details.json()["data"]['location']["num_reviews"]) > 0:
                review_dates.append(details.json()["data"]['location']["photo"]["published_date"])
            else:
                review_dates.append("")
            
    data = {
        'Location' : City
        "Name" : res_name,
        "Address" : res_add,
        "Reviews" : review_dates
    
    
    }


df = pd.DataFrame(data)

df.to_csv("Trip_Advisor.csv")
    
    
    

In [82]:
data_csv = Path('Input_data/GUF_1000m.csv')
drink_df = pd.read_csv(data_csv)

Places_df = pd.DataFrame(drink_df)

ON_df = Places_df[Places_df['province'] == 'ON']


#Places_df
#list(ON_df.columns)




In [ ]:
#REFERENCE TA REVIEWS API

Restaurants_df = pd.read_csv('output_data/restaurants.csv')

url = f"https://api.content.tripadvisor.com/api/v1/location/4556927/reviews?key={Trip_advisor_key}&language=en"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers).json()

#response['data']    ['rating']   

pprint(response)

In [ ]:
#REFERENCE TA lOCATION API

url = f"https://api.content.tripadvisor.com/api/v1/location/search?key={Trip_advisor_key}&searchQuery=Mom and Pops&language=en"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)
    
print(response.text)

In [ ]:
#REFERENCE GEOGRAPY API

radius = 1000

params = {
    'categories': 'catering.restaurant',
    'apiKey': geoapify_key,
}

base_url = 'https://api.geoapify.com/v2/places'

latitude = 43.6503
longitude = -79.3596

params['filter'] = f'circle:{longitude},{latitude},{radius}'
params['bias'] = f'proximity:{longitude},{latitude}'

response = requests.get(base_url, params=params).json()

pprint(response)